In [5]:
import os
from tensorflow import keras
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
#from keras.models import load_model,save_model
import numpy as np
import urllib.request, urllib.error, json

In [9]:
class roopre:
    IMAGESIZE=(224,224)
    INPUTLAYERSIZE=IMAGESIZE + (3,)
    
    def __init__(self):
        trainingimagesdata = ImageDataGenerator().flow_from_directory("./train_dataset",target_size = (224,224))
        self.num_classes = trainingimagesdata.num_classes
        self.ml_class_names = self.__get_class_lookup(trainingimagesdata)
    def __get_class_lookup(self,training_image_data):
        class_labels = [None]*training_image_data.num_classes
        class_names = training_image_data.class_indices.keys()
        for classname in class_names:
            class_labels[training_image_data.class_indices[classname]] = classname
        return class_labels
    def __define_model(self):
        #print("MLFORKIDS: Defining the layers to include in your neural network")
        model = Sequential([
            # input layer is resizing all images to save having to do that in a manual pre-processing step
            Rescaling(1/127, input_shape=(224, 224, 3)),
            # using an existing pre-trained model as an untrainable main layer
            hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/4"),
            #
            Dropout(rate=0.2),
            #
            Dense(self.num_classes)
        ])
        model.build((None,) + (224, 224, 3))

        # model compile parameters copied from tutorial at https://www.tensorflow.org/hub/tutorials/tf2_image_retraining
        model.compile(
            optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9),
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
            metrics=['accuracy'])

        return model
        
    def train_model(self):
        trainingimagesdata = ImageDataGenerator().flow_from_directory("./train_dataset",target_size = (224,224))
        self.ml_model = self.__define_model()
        if trainingimagesdata.batch_size > trainingimagesdata.samples:
            trainingimagesdata.batch_size = trainingimagesdata.samples
        steps_per_epoch = trainingimagesdata.samples // trainingimagesdata.batch_size
        epochs = 8
        if trainingimagesdata.samples > 55:
            epochs = 30
            #epochs = 1
        self.ml_model.fit(trainingimagesdata, epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=2)
        print("MLFORKIDS: Model training complete")
    def prediction(self, image_location: str):
        if hasattr(self, "ml_model") == False:
            raise RuntimeError("Machine learning model has not been trained for this project")
        testimg = image.load_img(image_location, target_size=(224,224))
        testimg = image.img_to_array(testimg)
        testimg = np.expand_dims(testimg, axis=0)
        predictions = self.ml_model.predict(testimg)
        topprediction = predictions[0]
        topanswer = np.argmax(topprediction)
        return self.ml_class_names[topanswer]
            
    def md_save(self):
        self.ml_model.save("test_model")
    def md_load(self):
        self.ml_model = keras.models.load_model("test_model")
        

In [10]:
rp = roopre()

Found 1760 images belonging to 20 classes.


In [11]:
rp.train_model()

Found 1760 images belonging to 20 classes.
Epoch 1/30
55/55 - 42s - loss: 2.7881 - accuracy: 0.3580
Epoch 2/30
55/55 - 38s - loss: 1.6803 - accuracy: 0.6631
Epoch 3/30
55/55 - 37s - loss: 1.3790 - accuracy: 0.7545
Epoch 4/30
55/55 - 37s - loss: 1.2704 - accuracy: 0.8011
Epoch 5/30
55/55 - 37s - loss: 1.1970 - accuracy: 0.8148
Epoch 6/30
55/55 - 37s - loss: 1.1565 - accuracy: 0.8432
Epoch 7/30
55/55 - 40s - loss: 1.0877 - accuracy: 0.8722
Epoch 8/30
55/55 - 38s - loss: 1.0742 - accuracy: 0.8767
Epoch 9/30
55/55 - 39s - loss: 1.0395 - accuracy: 0.8926
Epoch 10/30
55/55 - 38s - loss: 1.0113 - accuracy: 0.9170
Epoch 11/30
55/55 - 37s - loss: 1.0037 - accuracy: 0.9142
Epoch 12/30
55/55 - 35s - loss: 0.9737 - accuracy: 0.9222
Epoch 13/30
55/55 - 39s - loss: 0.9801 - accuracy: 0.9182
Epoch 14/30


KeyboardInterrupt: 

In [12]:
rp.md_save()

In [13]:
rp.md_load()

In [14]:
path = "./test_dataset/"

In [63]:
name = "라면"


In [64]:
print(rp.prediction(path+name+"/"+name+" 사진_0.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_1.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_2.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_3.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_4.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_5.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_6.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_7.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_8.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_9.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_10.jpg"))
print(rp.prediction(path+name+"/"+name+" 사진_11.jpg"))

라면
카레
토스트
떡볶이
김치찌개
라면
토스트
김치볶음밥
카레
김치
잔치국수
김치
